# TIC-TAC-TOE

![TIC-TAC-TOE](./images/tic-tac-toe.png)

In [2]:
#######################################################################
# Copyright (C)                                                       #
# 2016 Shangtong Zhang(zhangshangtong.cpp@gmail.com)                  #
# 2016 Jan Hakenberg(jan.hakenberg@gmail.com)                         #
# 2016 Tian Jun(tianjun.cpp@gmail.com)                                #
# 2016 Kenta Shimada(hyperkentakun@gmail.com)                         #
# Permission given to modify the code as long as you keep this        #
# declaration at the top                                              #
#######################################################################

from __future__ import print_function
import numpy as np
import pickle

BOARD_ROWS = 3
BOARD_COLS = 3
BOARD_SIZE = BOARD_ROWS * BOARD_COLS

class State:
    def __init__(self):
        # the board is represented by a n * n array,
        # 1 represents chessman of the player who moves first,
        # -1 represents chessman of another player
        # 0 represents empty position
        self.data = np.zeros((BOARD_ROWS, BOARD_COLS))
        self.winner = None
        self.hashVal = None
        self.end = None

    # calculate the hash value for one state, it's unique
    def getHash(self):
        if self.hashVal is None:
            self.hashVal = 0
            for i in self.data.reshape(BOARD_ROWS * BOARD_COLS):
                if i == -1:
                    i = 2
                self.hashVal = self.hashVal * 3 + i
        return int(self.hashVal)

    # determine whether a player has won the game, or it's a tie
    def isEnd(self):
        if self.end is not None:
            return self.end
        results = []
        # check row
        for i in range(0, BOARD_ROWS):
            results.append(np.sum(self.data[i, :]))
        # check columns
        for i in range(0, BOARD_COLS):
            results.append(np.sum(self.data[:, i]))

        # check diagonals
        results.append(0)
        for i in range(0, BOARD_ROWS):
            results[-1] += self.data[i, i]
        results.append(0)
        for i in range(0, BOARD_ROWS):
            results[-1] += self.data[i, BOARD_ROWS - 1 - i]

        for result in results:
            if result == 3:
                self.winner = 1
                self.end = True
                return self.end
            if result == -3:
                self.winner = -1
                self.end = True
                return self.end

        # whether it's a tie
        sum = np.sum(np.abs(self.data))
        if sum == BOARD_ROWS * BOARD_COLS:
            self.winner = 0
            self.end = True
            return self.end

        # game is still going on
        self.end = False
        return self.end

    # @symbol 1 or -1
    # put chessman symbol in position (i, j)
    def nextState(self, i, j, symbol):
        newState = State()
        newState.data = np.copy(self.data)
        newState.data[i, j] = symbol
        return newState

    # print the board
    def show(self):
        for i in range(0, BOARD_ROWS):
            print('-------------')
            out = '| '
            for j in range(0, BOARD_COLS):
                if self.data[i, j] == 1:
                    token = '*'
                if self.data[i, j] == 0:
                    token = '0'
                if self.data[i, j] == -1:
                    token = 'x'
                out += token + ' | '
            print(out)
        print('-------------')

def getAllStatesImpl(currentState, currentSymbol, allStates):
    for i in range(0, BOARD_ROWS):
        for j in range(0, BOARD_COLS):
            if currentState.data[i][j] == 0:
                newState = currentState.nextState(i, j, currentSymbol)
                newHash = newState.getHash()
                if newHash not in allStates.keys():
                    isEnd = newState.isEnd()
                    allStates[newHash] = (newState, isEnd)
                    if not isEnd:
                        getAllStatesImpl(newState, -currentSymbol, allStates)

def getAllStates():
    currentSymbol = 1
    currentState = State()
    allStates = dict()
    allStates[currentState.getHash()] = (currentState, currentState.isEnd())
    getAllStatesImpl(currentState, currentSymbol, allStates)
    return allStates

# all possible board configurations
allStates = getAllStates()

class Judger:
    # @player1: player who will move first, its chessman will be 1
    # @player2: another player with chessman -1
    # @feedback: if True, both players will receive rewards when game is end
    def __init__(self, player1, player2, feedback=True):
        self.p1 = player1
        self.p2 = player2
        self.feedback = feedback
        self.currentPlayer = None
        self.p1Symbol = 1
        self.p2Symbol = -1
        self.p1.setSymbol(self.p1Symbol)
        self.p2.setSymbol(self.p2Symbol)
        self.currentState = State()
        self.allStates = allStates

    # give reward to two players
    def giveReward(self):
        if self.currentState.winner == self.p1Symbol:
            self.p1.feedReward(1)
            self.p2.feedReward(0)
        elif self.currentState.winner == self.p2Symbol:
            self.p1.feedReward(0)
            self.p2.feedReward(1)
        else:
            self.p1.feedReward(0.1)
            self.p2.feedReward(0.5)

    def feedCurrentState(self):
        self.p1.feedState(self.currentState)
        self.p2.feedState(self.currentState)

    def reset(self):
        self.p1.reset()
        self.p2.reset()
        self.currentState = State()
        self.currentPlayer = None

    # @show: if True, print each board during the game
    def play(self, show=False):
        self.reset()
        self.feedCurrentState()
        while True:
            # set current player
            if self.currentPlayer == self.p1:
                self.currentPlayer = self.p2
            else:
                self.currentPlayer = self.p1
            if show:
                self.currentState.show()
            [i, j, symbol] = self.currentPlayer.takeAction()
            self.currentState = self.currentState.nextState(i, j, symbol)
            hashValue = self.currentState.getHash()
            self.currentState, isEnd = self.allStates[hashValue]
            self.feedCurrentState()
            if isEnd:
                if self.feedback:
                    self.giveReward()
                return self.currentState.winner

# AI player
class Player:
    # @stepSize: step size to update estimations
    # @exploreRate: possibility to explore
    def __init__(self, stepSize = 0.1, exploreRate=0.1):
        self.allStates = allStates
        self.estimations = dict()
        self.stepSize = stepSize
        self.exploreRate = exploreRate
        self.states = []

    def reset(self):
        self.states = []

    def setSymbol(self, symbol):
        self.symbol = symbol
        for hash in self.allStates.keys():
            (state, isEnd) = self.allStates[hash]
            if isEnd:
                if state.winner == self.symbol:
                    self.estimations[hash] = 1.0
                else:
                    self.estimations[hash] = 0
            else:
                self.estimations[hash] = 0.5

    # accept a state
    def feedState(self, state):
        self.states.append(state)

    # update estimation according to reward
    def feedReward(self, reward):
        if len(self.states) == 0:
            return
        self.states = [state.getHash() for state in self.states]
        target = reward
        for latestState in reversed(self.states):
            value = self.estimations[latestState] + self.stepSize * (target - self.estimations[latestState])
            self.estimations[latestState] = value
            target = value
        self.states = []

    # determine next action
    def takeAction(self):
        state = self.states[-1]
        nextStates = []
        nextPositions = []
        for i in range(BOARD_ROWS):
            for j in range(BOARD_COLS):
                if state.data[i, j] == 0:
                    nextPositions.append([i, j])
                    nextStates.append(state.nextState(i, j, self.symbol).getHash())
        if np.random.binomial(1, self.exploreRate):
            np.random.shuffle(nextPositions)
            # Not sure if truncating is the best way to deal with exploratory step
            # Maybe it's better to only skip this step rather than forget all the history
            self.states = []
            action = nextPositions[0]
            action.append(self.symbol)
            return action

        values = []
        for hash, pos in zip(nextStates, nextPositions):
            values.append((self.estimations[hash], pos))
        np.random.shuffle(values)
        values.sort(key=lambda x: x[0], reverse=True)
        action = values[0][1]
        action.append(self.symbol)
        return action

    def savePolicy(self):
        fw = open('optimal_policy_' + str(self.symbol), 'wb')
        pickle.dump(self.estimations, fw)
        fw.close()

    def loadPolicy(self):
        fr = open('optimal_policy_' + str(self.symbol),'rb')
        self.estimations = pickle.load(fr)
        fr.close()

# human interface
# input a number to put a chessman
# | 1 | 2 | 3 |
# | 4 | 5 | 6 |
# | 7 | 8 | 9 |
class HumanPlayer:
    def __init__(self, stepSize = 0.1, exploreRate=0.1):
        self.symbol = None
        self.currentState = None
        return
    def reset(self):
        return
    def setSymbol(self, symbol):
        self.symbol = symbol
        return
    def feedState(self, state):
        self.currentState = state
        return
    def feedReward(self, reward):
        return
    def takeAction(self):
        data = int(input("Input your position:"))
        data -= 1
        i = data // int(BOARD_COLS)
        j = data % BOARD_COLS
        if self.currentState.data[i, j] != 0:
            return self.takeAction()
        return (i, j, self.symbol)

def train(epochs=20000):
    player1 = Player()
    player2 = Player()
    judger = Judger(player1, player2)
    player1Win = 0.0
    player2Win = 0.0
    for i in range(0, epochs):
        print("Epoch", i)
        winner = judger.play()
        if winner == 1:
            player1Win += 1
        if winner == -1:
            player2Win += 1
        judger.reset()
    print(player1Win / epochs)
    print(player2Win / epochs)
    player1.savePolicy()
    player2.savePolicy()

def compete(turns=500):
    player1 = Player(exploreRate=0)
    player2 = Player(exploreRate=0)
    judger = Judger(player1, player2, False)
    player1.loadPolicy()
    player2.loadPolicy()
    player1Win = 0.0
    player2Win = 0.0
    for i in range(0, turns):
        print("Epoch", i)
        winner = judger.play()
        if winner == 1:
            player1Win += 1
        if winner == -1:
            player2Win += 1
        judger.reset()
    print(player1Win / turns)
    print(player2Win / turns)

def play():
    while True:
        player1 = Player(exploreRate=0)
        player2 = HumanPlayer()
        judger = Judger(player1, player2, False)
        player1.loadPolicy()
        winner = judger.play(True)
        if winner == player2.symbol:
            print("Win!")
        elif winner == player1.symbol:
            print("Lose!")
        else:
            print("Tie!")

train()
compete()
play()


Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29
Epoch 30
Epoch 31
Epoch 32
Epoch 33
Epoch 34
Epoch 35
Epoch 36
Epoch 37
Epoch 38
Epoch 39
Epoch 40
Epoch 41
Epoch 42
Epoch 43
Epoch 44
Epoch 45
Epoch 46
Epoch 47
Epoch 48
Epoch 49
Epoch 50
Epoch 51
Epoch 52
Epoch 53
Epoch 54
Epoch 55
Epoch 56
Epoch 57
Epoch 58
Epoch 59
Epoch 60
Epoch 61
Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
Epoch 100
Epoch 101
Epoch 102
Epoch 103
Epoch 104
Epoch 105
Epoch 106
Epoch 107
Epoch 108
Epoch 109
Epoch 110


Epoch 898
Epoch 899
Epoch 900
Epoch 901
Epoch 902
Epoch 903
Epoch 904
Epoch 905
Epoch 906
Epoch 907
Epoch 908
Epoch 909
Epoch 910
Epoch 911
Epoch 912
Epoch 913
Epoch 914
Epoch 915
Epoch 916
Epoch 917
Epoch 918
Epoch 919
Epoch 920
Epoch 921
Epoch 922
Epoch 923
Epoch 924
Epoch 925
Epoch 926
Epoch 927
Epoch 928
Epoch 929
Epoch 930
Epoch 931
Epoch 932
Epoch 933
Epoch 934
Epoch 935
Epoch 936
Epoch 937
Epoch 938
Epoch 939
Epoch 940
Epoch 941
Epoch 942
Epoch 943
Epoch 944
Epoch 945
Epoch 946
Epoch 947
Epoch 948
Epoch 949
Epoch 950
Epoch 951
Epoch 952
Epoch 953
Epoch 954
Epoch 955
Epoch 956
Epoch 957
Epoch 958
Epoch 959
Epoch 960
Epoch 961
Epoch 962
Epoch 963
Epoch 964
Epoch 965
Epoch 966
Epoch 967
Epoch 968
Epoch 969
Epoch 970
Epoch 971
Epoch 972
Epoch 973
Epoch 974
Epoch 975
Epoch 976
Epoch 977
Epoch 978
Epoch 979
Epoch 980
Epoch 981
Epoch 982
Epoch 983
Epoch 984
Epoch 985
Epoch 986
Epoch 987
Epoch 988
Epoch 989
Epoch 990
Epoch 991
Epoch 992
Epoch 993
Epoch 994
Epoch 995
Epoch 996
Epoch 997


Epoch 1779
Epoch 1780
Epoch 1781
Epoch 1782
Epoch 1783
Epoch 1784
Epoch 1785
Epoch 1786
Epoch 1787
Epoch 1788
Epoch 1789
Epoch 1790
Epoch 1791
Epoch 1792
Epoch 1793
Epoch 1794
Epoch 1795
Epoch 1796
Epoch 1797
Epoch 1798
Epoch 1799
Epoch 1800
Epoch 1801
Epoch 1802
Epoch 1803
Epoch 1804
Epoch 1805
Epoch 1806
Epoch 1807
Epoch 1808
Epoch 1809
Epoch 1810
Epoch 1811
Epoch 1812
Epoch 1813
Epoch 1814
Epoch 1815
Epoch 1816
Epoch 1817
Epoch 1818
Epoch 1819
Epoch 1820
Epoch 1821
Epoch 1822
Epoch 1823
Epoch 1824
Epoch 1825
Epoch 1826
Epoch 1827
Epoch 1828
Epoch 1829
Epoch 1830
Epoch 1831
Epoch 1832
Epoch 1833
Epoch 1834
Epoch 1835
Epoch 1836
Epoch 1837
Epoch 1838
Epoch 1839
Epoch 1840
Epoch 1841
Epoch 1842
Epoch 1843
Epoch 1844
Epoch 1845
Epoch 1846
Epoch 1847
Epoch 1848
Epoch 1849
Epoch 1850
Epoch 1851
Epoch 1852
Epoch 1853
Epoch 1854
Epoch 1855
Epoch 1856
Epoch 1857
Epoch 1858
Epoch 1859
Epoch 1860
Epoch 1861
Epoch 1862
Epoch 1863
Epoch 1864
Epoch 1865
Epoch 1866
Epoch 1867
Epoch 1868
Epoch 1869

Epoch 2649
Epoch 2650
Epoch 2651
Epoch 2652
Epoch 2653
Epoch 2654
Epoch 2655
Epoch 2656
Epoch 2657
Epoch 2658
Epoch 2659
Epoch 2660
Epoch 2661
Epoch 2662
Epoch 2663
Epoch 2664
Epoch 2665
Epoch 2666
Epoch 2667
Epoch 2668
Epoch 2669
Epoch 2670
Epoch 2671
Epoch 2672
Epoch 2673
Epoch 2674
Epoch 2675
Epoch 2676
Epoch 2677
Epoch 2678
Epoch 2679
Epoch 2680
Epoch 2681
Epoch 2682
Epoch 2683
Epoch 2684
Epoch 2685
Epoch 2686
Epoch 2687
Epoch 2688
Epoch 2689
Epoch 2690
Epoch 2691
Epoch 2692
Epoch 2693
Epoch 2694
Epoch 2695
Epoch 2696
Epoch 2697
Epoch 2698
Epoch 2699
Epoch 2700
Epoch 2701
Epoch 2702
Epoch 2703
Epoch 2704
Epoch 2705
Epoch 2706
Epoch 2707
Epoch 2708
Epoch 2709
Epoch 2710
Epoch 2711
Epoch 2712
Epoch 2713
Epoch 2714
Epoch 2715
Epoch 2716
Epoch 2717
Epoch 2718
Epoch 2719
Epoch 2720
Epoch 2721
Epoch 2722
Epoch 2723
Epoch 2724
Epoch 2725
Epoch 2726
Epoch 2727
Epoch 2728
Epoch 2729
Epoch 2730
Epoch 2731
Epoch 2732
Epoch 2733
Epoch 2734
Epoch 2735
Epoch 2736
Epoch 2737
Epoch 2738
Epoch 2739

Epoch 3480
Epoch 3481
Epoch 3482
Epoch 3483
Epoch 3484
Epoch 3485
Epoch 3486
Epoch 3487
Epoch 3488
Epoch 3489
Epoch 3490
Epoch 3491
Epoch 3492
Epoch 3493
Epoch 3494
Epoch 3495
Epoch 3496
Epoch 3497
Epoch 3498
Epoch 3499
Epoch 3500
Epoch 3501
Epoch 3502
Epoch 3503
Epoch 3504
Epoch 3505
Epoch 3506
Epoch 3507
Epoch 3508
Epoch 3509
Epoch 3510
Epoch 3511
Epoch 3512
Epoch 3513
Epoch 3514
Epoch 3515
Epoch 3516
Epoch 3517
Epoch 3518
Epoch 3519
Epoch 3520
Epoch 3521
Epoch 3522
Epoch 3523
Epoch 3524
Epoch 3525
Epoch 3526
Epoch 3527
Epoch 3528
Epoch 3529
Epoch 3530
Epoch 3531
Epoch 3532
Epoch 3533
Epoch 3534
Epoch 3535
Epoch 3536
Epoch 3537
Epoch 3538
Epoch 3539
Epoch 3540
Epoch 3541
Epoch 3542
Epoch 3543
Epoch 3544
Epoch 3545
Epoch 3546
Epoch 3547
Epoch 3548
Epoch 3549
Epoch 3550
Epoch 3551
Epoch 3552
Epoch 3553
Epoch 3554
Epoch 3555
Epoch 3556
Epoch 3557
Epoch 3558
Epoch 3559
Epoch 3560
Epoch 3561
Epoch 3562
Epoch 3563
Epoch 3564
Epoch 3565
Epoch 3566
Epoch 3567
Epoch 3568
Epoch 3569
Epoch 3570

...

Epoch 17700
Epoch 17701
Epoch 17702
Epoch 17703
Epoch 17704
Epoch 17705
Epoch 17706
Epoch 17707
Epoch 17708
Epoch 17709
Epoch 17710
Epoch 17711
Epoch 17712
Epoch 17713
Epoch 17714
Epoch 17715
Epoch 17716
Epoch 17717
Epoch 17718
Epoch 17719
Epoch 17720
Epoch 17721
Epoch 17722
Epoch 17723
Epoch 17724
Epoch 17725
Epoch 17726
Epoch 17727
Epoch 17728
Epoch 17729
Epoch 17730
Epoch 17731
Epoch 17732
Epoch 17733
Epoch 17734
Epoch 17735
Epoch 17736
Epoch 17737
Epoch 17738
Epoch 17739
Epoch 17740
Epoch 17741
Epoch 17742
Epoch 17743
Epoch 17744
Epoch 17745
Epoch 17746
Epoch 17747
Epoch 17748
Epoch 17749
Epoch 17750
Epoch 17751
Epoch 17752
Epoch 17753
Epoch 17754
Epoch 17755
Epoch 17756
Epoch 17757
Epoch 17758
Epoch 17759
Epoch 17760
Epoch 17761
Epoch 17762
Epoch 17763
Epoch 17764
Epoch 17765
Epoch 17766
Epoch 17767
Epoch 17768
Epoch 17769
Epoch 17770
Epoch 17771
Epoch 17772
Epoch 17773
Epoch 17774
Epoch 17775
Epoch 17776
Epoch 17777
Epoch 17778
Epoch 17779
Epoch 17780
Epoch 17781
Epoch 17782
Epoc

Epoch 18472
Epoch 18473
Epoch 18474
Epoch 18475
Epoch 18476
Epoch 18477
Epoch 18478
Epoch 18479
Epoch 18480
Epoch 18481
Epoch 18482
Epoch 18483
Epoch 18484
Epoch 18485
Epoch 18486
Epoch 18487
Epoch 18488
Epoch 18489
Epoch 18490
Epoch 18491
Epoch 18492
Epoch 18493
Epoch 18494
Epoch 18495
Epoch 18496
Epoch 18497
Epoch 18498
Epoch 18499
Epoch 18500
Epoch 18501
Epoch 18502
Epoch 18503
Epoch 18504
Epoch 18505
Epoch 18506
Epoch 18507
Epoch 18508
Epoch 18509
Epoch 18510
Epoch 18511
Epoch 18512
Epoch 18513
Epoch 18514
Epoch 18515
Epoch 18516
Epoch 18517
Epoch 18518
Epoch 18519
Epoch 18520
Epoch 18521
Epoch 18522
Epoch 18523
Epoch 18524
Epoch 18525
Epoch 18526
Epoch 18527
Epoch 18528
Epoch 18529
Epoch 18530
Epoch 18531
Epoch 18532
Epoch 18533
Epoch 18534
Epoch 18535
Epoch 18536
Epoch 18537
Epoch 18538
Epoch 18539
Epoch 18540
Epoch 18541
Epoch 18542
Epoch 18543
Epoch 18544
Epoch 18545
Epoch 18546
Epoch 18547
Epoch 18548
Epoch 18549
Epoch 18550
Epoch 18551
Epoch 18552
Epoch 18553
Epoch 18554
Epoc

Epoch 19268
Epoch 19269
Epoch 19270
Epoch 19271
Epoch 19272
Epoch 19273
Epoch 19274
Epoch 19275
Epoch 19276
Epoch 19277
Epoch 19278
Epoch 19279
Epoch 19280
Epoch 19281
Epoch 19282
Epoch 19283
Epoch 19284
Epoch 19285
Epoch 19286
Epoch 19287
Epoch 19288
Epoch 19289
Epoch 19290
Epoch 19291
Epoch 19292
Epoch 19293
Epoch 19294
Epoch 19295
Epoch 19296
Epoch 19297
Epoch 19298
Epoch 19299
Epoch 19300
Epoch 19301
Epoch 19302
Epoch 19303
Epoch 19304
Epoch 19305
Epoch 19306
Epoch 19307
Epoch 19308
Epoch 19309
Epoch 19310
Epoch 19311
Epoch 19312
Epoch 19313
Epoch 19314
Epoch 19315
Epoch 19316
Epoch 19317
Epoch 19318
Epoch 19319
Epoch 19320
Epoch 19321
Epoch 19322
Epoch 19323
Epoch 19324
Epoch 19325
Epoch 19326
Epoch 19327
Epoch 19328
Epoch 19329
Epoch 19330
Epoch 19331
Epoch 19332
Epoch 19333
Epoch 19334
Epoch 19335
Epoch 19336
Epoch 19337
Epoch 19338
Epoch 19339
Epoch 19340
Epoch 19341
Epoch 19342
Epoch 19343
Epoch 19344
Epoch 19345
Epoch 19346
Epoch 19347
Epoch 19348
Epoch 19349
Epoch 19350
Epoc

Epoch 62
Epoch 63
Epoch 64
Epoch 65
Epoch 66
Epoch 67
Epoch 68
Epoch 69
Epoch 70
Epoch 71
Epoch 72
Epoch 73
Epoch 74
Epoch 75
Epoch 76
Epoch 77
Epoch 78
Epoch 79
Epoch 80
Epoch 81
Epoch 82
Epoch 83
Epoch 84
Epoch 85
Epoch 86
Epoch 87
Epoch 88
Epoch 89
Epoch 90
Epoch 91
Epoch 92
Epoch 93
Epoch 94
Epoch 95
Epoch 96
Epoch 97
Epoch 98
Epoch 99
Epoch 100
Epoch 101
Epoch 102
Epoch 103
Epoch 104
Epoch 105
Epoch 106
Epoch 107
Epoch 108
Epoch 109
Epoch 110
Epoch 111
Epoch 112
Epoch 113
Epoch 114
Epoch 115
Epoch 116
Epoch 117
Epoch 118
Epoch 119
Epoch 120
Epoch 121
Epoch 122
Epoch 123
Epoch 124
Epoch 125
Epoch 126
Epoch 127
Epoch 128
Epoch 129
Epoch 130
Epoch 131
Epoch 132
Epoch 133
Epoch 134
Epoch 135
Epoch 136
Epoch 137
Epoch 138
Epoch 139
Epoch 140
Epoch 141
Epoch 142
Epoch 143
Epoch 144
Epoch 145
Epoch 146
Epoch 147
Epoch 148
Epoch 149
Epoch 150
Epoch 151
Epoch 152
Epoch 153
Epoch 154
Epoch 155
Epoch 156
Epoch 157
Epoch 158
Epoch 159
Epoch 160
Epoch 161
Epoch 162
Epoch 163
Epoch 164
Epoch 16

KeyboardInterrupt: 

In [ ]:
# page 32, A sequence of tic-tac-toe moves.
# page 105, state value estimates.